# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Разделим-данные-на-features-и-target" data-toc-modified-id="Разделим-данные-на-features-и-target-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Разделим данные на <code>features</code> и <code>target</code></a></span></li><li><span><a href="#Разделим-выборку-на-test-и-train" data-toc-modified-id="Разделим-выборку-на-test-и-train-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Разделим выборку на test и train</a></span></li><li><span><a href="#Обучим-модель-на-непреобразованных-данных" data-toc-modified-id="Обучим-модель-на-непреобразованных-данных-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Обучим модель на непреобразованных данных</a></span></li><li><span><a href="#Преобразуем-данные" data-toc-modified-id="Преобразуем-данные-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Преобразуем данные</a></span></li><li><span><a href="#Обучим-и-протестируем-модель-на-преобразованных-данных" data-toc-modified-id="Обучим-и-протестируем-модель-на-преобразованных-данных-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Обучим и протестируем модель на преобразованных данных</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
def basic_data_info(data):
    print('Первые пять строк:')
    display(data.head())
    print('\nИнформация:')
    print(data.info())
    print('\nНаличие пропусков:')
    print(data.isna().sum())
    print('\nНаличие дубликатов:')
    print(data.duplicated().sum())
    print('\nСтатистическое описание данных:')
    print(data.describe())

In [4]:
basic_data_info(data)

Первые пять строк:


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



Информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None

Наличие пропусков:
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Наличие дубликатов:
153

Статистическое описание данных:
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
mean      0.499000    30.952800  39916.360000     1.194200           0.148000
std       0.500049     8.440807   9900.083569   

Мы загрузили данные в датафрейм `data`. В данных содержится 5000 строк-наблюдений. Видим в данных следующие признаки:
- `Пол`: категориальный признак типа `int64`, принимающий значения `1` и `0`. Пропусков нет.
- `Возраст`: числовой признак типа `float64`, принимает целочисленные значения от 18 до 65 включительно. Пропусков нет.
- `Зарплата`: числовой признак типа `float64`, принимает значения в диапазоне [5300, 79000]. Пропусков нет.
- `Члены семьи`: числовой признак типа `int64`, принимает целочисленные значения в диапазоне [0, 6]. Пропусков нет.
- `Страховые выплаты`: целевой числовой признак типа `int64`, принимает значения в диапазоне [0, 5]. Пропусков нет.

В данных есть 153 строки-дубликата.


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** Пусть матрица $Z = XP$. Запишем формулу обучения для новой матрицы:

$$w_z = (Z^T Z)^{-1} Z^T y = ((XP)^T (XP))^{-1} (XP)^T y$$

Упростим выражение, используя следующие свойства:
- $A(BC) = (AB)C$
- $(AB)^T = B^T A^T$
- $(AB)^{-1} = B^{-1} A^{-1}$

$$w_z = ((XP)^T (XP))^{-1} (XP)^T y = (((XP)^T X)P)^{-1} (XP)^T y = P^{-1} ((XP)^T X)^{-1} (XP)^T y = P^{-1} (P^T X^T X)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Итого, получили, что $w_z = P^{-1} w$. Предсказания будут рассчитываться следующим образом:
$$a_z = Z w_z = XP P^{-1} w = Xw = a$$

Получаем, что новые предсказания равны старым.

Мы доказали, что при умножении признаков на обратимую матрицу, предсказания не изменятся.

## Алгоритм преобразования

**Алгоритм**
1. Генерируем случайную обратимую матрицу размерности (n x n).
2. Обучаем линейную регрессию.
3. Предсказываем значение целевого признака.

**Обоснование**

В предыдущем пункте доказали, что при умножении матрицы признаков на обратимую матрицу качество линейной регрессии не меняется, поэтому умножение признаков на обратимую матрицу можно использовать длся преобразования признаков без ухудшения качества работы модели.

## Проверка алгоритма

### Разделим данные на `features` и `target`

In [5]:
features = data.drop(columns=['Страховые выплаты']).values

In [6]:
features

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [7]:
target = data['Страховые выплаты'].values

In [8]:
target

array([0, 1, 0, ..., 0, 0, 0])

### Разделим выборку на test и train

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=42)

### Обучим модель на непреобразованных данных

In [10]:
model = LinearRegression()

In [11]:
model.fit(features_train, target_train)

LinearRegression()

In [12]:
predictions = model.predict(features_test)

In [13]:
r2 = r2_score(target_test, predictions)

In [14]:
print('R2-score = {:.3f}'.format(r2))

R2-score = 0.423


### Преобразуем данные
Создадим случайную матрицу P нужной размерности и проверим, что она обратимая.

In [15]:
features.shape[1]

4

In [16]:
np.random.seed(42)

In [17]:
def get_rand_matrix():
    det = 0
    while det == 0:
        matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
        det = np.linalg.det(matrix)
    return matrix

In [18]:
P = get_rand_matrix()

In [19]:
P

array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986],
       [-0.23415337, -0.23413696,  1.57921282,  0.76743473],
       [-0.46947439,  0.54256004, -0.46341769, -0.46572975],
       [ 0.24196227, -1.91328024, -1.72491783, -0.56228753]])

In [20]:
P_inv = np.linalg.inv(P)

In [21]:
P_inv

array([[-0.33124857, -1.0333483 , -2.21070923, -0.47651024],
       [ 0.12864778, -0.47781825,  0.15271687, -0.43017975],
       [-0.51338817,  0.33588835, -0.8347692 , -0.24072429],
       [ 0.99462192,  0.15079316,  1.08985145,  0.21872535]])

Преобразуем наши признаки

In [22]:
features_train_transformed = np.dot(features_train, P)

In [23]:
features_test_transformed = np.dot(features_test, P)

### Обучим и протестируем модель на преобразованных данных

In [24]:
model_transformed = LinearRegression()

In [25]:
model_transformed.fit(features_train_transformed, target_train)

LinearRegression()

In [26]:
predictions_transformed = model_transformed.predict(features_test_transformed)

In [27]:
r2_transformed = r2_score(target_test, predictions_transformed)

In [28]:
print('R2-score = {:.3f}'.format(r2_transformed))

R2-score = 0.423


## Вывод

Был выполнен проект по теме "Защита персональных данных клиентов". Выполнение проекта проводилось в несколько этапов:
1. Загрузка данных в датафрейм `data`. Предварительное изучение данных.
2. Теоретическое доказательство утверждения, что качество линейной регрессии не изменится при умножении матрицы признаков $X$ на обратимую матрицу $P$. Было доказано, что в исходной и преобразованной задаче параметры равны, т.е. минимум функции потерь будет достигаться при одинаковых значениях.
3. Был предложен алгоритм преобразования данных, в рамках которого данные преобразовываются домножением матрицы признаков на обратимую матрицу размерности (n x n), где n - это количество признаков в данных.
4. Были обучены две модели: модель линейной регресси на непреобразованных данных и модель линейной регрессии на преобразованных данных. Было показано, что значения R2-score равны.